# Demonstration of the SCF Voting Mechanism for PoC Purposes

*Danilo Lessa Bernardineli (BlockScience), July 2023*

On this Notebook, we'll provide an stylized demonstration for how
Neural Governance could be implemented along with four of its modules: Quorum Voting Neuron, Trust Neuron, Voting History Neuron and Expertise Neuron. Detailed instructions can be found on the [SCF Voting Mechanism PoC Specification](https://hackmd.io/GMs8iB1MQEGsJirqqPm3NA) and [SDF Voting Mechanism PoC Design](https://hackmd.io/HzRrf1NtQ_a7nlSvX_stXg?both) documents.


As a testing benchmark, we'll also generate fictious datasets that
represents the Voting Users, their actions, their trust graph, and the
inputs for the Voting History / Expertise Neuron.

## Definitions

## Data Structures

## User Data Generation

## 

## Computing Voting Results